In [1]:
running = False
try:
    sock.close()
except:
    pass

In [2]:
from parser import public_construct, public_match

In [3]:
from utils import SocketReader

In [4]:
rmeote_offer = """v=0
o=- 4905540758590263656 2 IN IP4 127.0.0.1
s=-
t=0 0
a=group:BUNDLE 0
a=extmap-allow-mixed
a=msid-semantic: WMS
m=application 51796 UDP/DTLS/SCTP webrtc-datachannel
c=IN IP4 34.42.107.192
a=candidate:1432319694 1 udp 2113937151 5aefde91-14a2-4eb7-b69d-c3d5a39338d2.local 51796 typ host generation 0 network-cost 999
a=candidate:2690825085 1 udp 1677729535 34.42.107.192 51796 typ srflx raddr 0.0.0.0 rport 0 generation 0 network-cost 999
a=ice-ufrag:28Hw
a=ice-pwd:yL3N6bzhVdea8Yf5cbpgYz2L
a=ice-options:trickle
a=fingerprint:sha-256 EB:67:CB:B8:04:53:FA:94:41:77:1F:4D:E1:A4:4E:05:A7:DB:94:67:D4:2C:31:A1:F2:7B:B6:DC:3E:B7:7B:D8
a=setup:actpass
a=mid:0
a=sctp-port:5000
a=max-message-size:262144

"""

In [5]:
for line in rmeote_offer.splitlines():
    if 'a=candidate:' in line and '.local' not in line:
        res = line.split(' ')
        target_ip = res[4]
        target_port = int(res[5])
    if 'a=ice-ufrag:' in line:
        remote_ufrag = line.split(':')[1]
    if 'a=ice-pwd:' in line:
        remote_pwd = line.split(':')[1]

target_ip_num = 0
for i, seg in enumerate(target_ip.split('.')):
    target_ip_num += int(seg) * (256**(3-i))

target_ip, target_port, remote_ufrag, remote_pwd

('34.42.107.192', 51796, '28Hw', 'yL3N6bzhVdea8Yf5cbpgYz2L')

In [6]:
import random, string
len_remote_ufrag = len(remote_ufrag)
# len_remote_ufrag = 4 # chrome
local_ufrag = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16 - len_remote_ufrag - 1))
local_pwd = 'AgLw44VDUK531eYVCMpZMAjF'
local_ufrag

'JIqQD69kEv6'

In [7]:
from stun import do_stun
import random
sport = random.randrange(30000, 50000)
ip, port = do_stun(sport)
f"found {ip}:{port} while source port is {sport}"

'found 143.178.72.250:46973 while source port is 46973'

In [8]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
import OpenSSL.crypto

own_cert_private_key = ec.generate_private_key(ec.SECP256R1())

own_cert_private_key_bytes = own_cert_private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)


pkey = OpenSSL.crypto.load_privatekey(OpenSSL.crypto.FILETYPE_PEM, own_cert_private_key_bytes)


cert_req = OpenSSL.crypto.X509Req()

cert_req.get_subject().CN = 'WebRTC'

cert = OpenSSL.crypto.X509()
cert.gmtime_adj_notBefore(0)
cert.gmtime_adj_notAfter(1 * 24 * 60 * 60)
cert.set_issuer(cert_req.get_subject())
cert.set_subject(cert_req.get_subject())
cert.set_pubkey(pkey)
cert.sign(pkey, 'sha256') 
print(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_TEXT, cert).decode('utf-8')) 
local_cert_sha = cert.digest('sha256').decode('utf-8')
local_cert_sha

Certificate:
    Data:
        Version: 1 (0x0)
        Serial Number:
                    Signature Algorithm: ecdsa-with-SHA256
        Issuer: CN=WebRTC
        Validity
            Not Before: Jun 18 20:42:34 2024 GMT
            Not After : Jun 19 20:42:34 2024 GMT
        Subject: CN=WebRTC
        Subject Public Key Info:
            Public Key Algorithm: id-ecPublicKey
                Public-Key: (256 bit)
                pub:
                    04:95:61:36:b0:55:3f:d4:9f:ec:36:cf:71:fd:86:
                    33:dd:5d:18:1c:7c:8f:a2:04:24:f6:5c:cf:3d:de:
                    fa:dc:a5:dc:53:f9:86:84:5c:25:03:cf:ce:f9:b8:
                    cb:3b:cd:69:25:6a:23:c4:e7:00:3a:97:22:ab:1d:
                    57:ba:36:1c:f6
                ASN1 OID: prime256v1
                NIST CURVE: P-256
    Signature Algorithm: ecdsa-with-SHA256
    Signature Value:
        30:45:02:21:00:af:76:71:6e:16:2f:51:a0:0d:73:49:58:28:
        62:11:80:ad:75:c3:ff:5d:3b:b0:81:9f:3e:af:1b:5e:e6:e6:
 

'24:6A:49:57:D6:22:F1:0B:0A:17:30:57:A3:53:64:11:C4:3A:98:AC:43:0A:D3:18:30:56:07:F2:8B:13:D8:19'

In [9]:
local_offer = f"""v=0
o=- 7407514470269221864 2 IN IP4 127.0.0.1
s=-
t=0 0
a=group:BUNDLE 0
a=extmap-allow-mixed
a=msid-semantic: WMS
m=application {port} UDP/DTLS/SCTP webrtc-datachannel
c=IN IP4 {ip}
a=candidate:3055715734 1 udp 1677729535 {ip} {port} typ srflx raddr 0.0.0.0 rport 0 generation 0 network-cost 999
a=ice-ufrag:{local_ufrag}
a=ice-pwd:{local_pwd}
a=ice-options:trickle
a=fingerprint:sha-256 {local_cert_sha}
a=setup:passive
a=mid:0
a=sctp-port:5000
a=max-message-size:262144
"""
print(local_offer)

v=0
o=- 7407514470269221864 2 IN IP4 127.0.0.1
s=-
t=0 0
a=group:BUNDLE 0
a=extmap-allow-mixed
a=msid-semantic: WMS
m=application 46973 UDP/DTLS/SCTP webrtc-datachannel
c=IN IP4 143.178.72.250
a=candidate:3055715734 1 udp 1677729535 143.178.72.250 46973 typ srflx raddr 0.0.0.0 rport 0 generation 0 network-cost 999
a=ice-ufrag:JIqQD69kEv6
a=ice-pwd:AgLw44VDUK531eYVCMpZMAjF
a=ice-options:trickle
a=fingerprint:sha-256 24:6A:49:57:D6:22:F1:0B:0A:17:30:57:A3:53:64:11:C4:3A:98:AC:43:0A:D3:18:30:56:07:F2:8B:13:D8:19
a=setup:passive
a=mid:0
a=sctp-port:5000
a=max-message-size:262144



In [10]:
stun_header_spec = """
2@zeroes
-14@message_type
-16@message_length
-32@magic_cookie
-96@transaction_id
"""

padding_0_attribute_spec = """
...(
    16==0x20?16@xor_mapped_ip(
        8@zeroes
        -8@family
        -16@port
        -32@ip
    )
    :16==0x06?16@username()
    :16==0x08?16@message_integrity(20B@hmac)
    :16==0x25?16@use_candidate()
    :16==0x8029?16@ice_controlled(8B@tie_breaker)
    :16==0x802a?16@ice_controlling(8B@tie_breaker)
    :16==0x24?16@priority(4B@priority)
    :16==0x8028?16@fingerprint(4B@crc32)
    :16@selector-16@data()
)
"""

import random
magic_cookie = 0x2112A442 # constant

In [11]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.hmac import HMAC

def get_message_integrity(msg, key):
    msg = bytearray(msg)
    msg[3] = len(msg) + 24 - 20
    hm = HMAC(key, hashes.SHA1())
    hm.update(msg)
    return hm.finalize()

def append_message_integrity(msg, key):
    hmac = get_message_integrity(msg, key)
    attrs = [{"message_integrity": {"hmac": hmac}, "_selector": 0x08}]
    msg = bytearray(msg)
    msg += public_construct(padding_0_attribute_spec, attrs)
    msg[3] = len(msg) - 20
    return msg

import binascii
def append_fingerprint(msg):
    msg = bytearray(msg)
    msg[3] = len(msg) + 8 - 20
    crc32 = binascii.crc32(msg) ^ 0x5354554e
    attrs = [{"fingerprint": {"crc32": crc32}, "_selector": 0x8028}]
    msg += public_construct(padding_0_attribute_spec, attrs)
    return msg
    


# v = """000100502112a4427139396c466f2b2f642f48650006000952556d333a4e6b4b36000000c0570004000003e7802a00084f02cc537c6985ff00250000002400046e001eff00080014ed4243520504f6eeefe90146a25142054ab03a4380280004f0c40a27"""
# msgm = bytes.fromhex(v)
# should_be = msgm[-28:-8].hex()
# print('should be', should_be)
# msgm = bytearray(msgm[:-8][:-24])
# key = b'7bjW8vqNNZPvdFDCa3IJ3J7E'
# print(v)
# print(append_fingerprint(append_message_integrity(msgm, key)).hex())


In [12]:
def create_response(transaction_id):
    attributes = [
        {'xor_mapped_ip': {'zeroes': 0x00,
       'family': 0x01,
       'port': target_port  ^ (magic_cookie >> 16),
       'ip': target_ip_num  ^ (magic_cookie)},
      '_selector': 0x0020}
    ]
    
    msg = public_construct(padding_0_attribute_spec, attributes)
    
    stun_header = {
        "zeroes": 0,
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        # |M |M |M|M|M|C|M|M|M|C|M|M|M|M|
        # |11|10|9|8|7|1|6|5|4|0|3|2|1|0|
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        "message_type": 0b00000100000001,
        "message_length": len(msg)+24,
        "magic_cookie": magic_cookie,
        "transaction_id": transaction_id,
    }
    
    msg = public_construct(stun_header_spec, stun_header) + msg
    msg = append_message_integrity(msg, local_pwd.encode('utf-8'))
    msg = append_fingerprint(msg)
    return msg

def create_request():
    attributes = [
        {"username": remote_ufrag+':'+local_ufrag, "_selector": 0x06},
        # {"use_candidate": "", "_selector": 0x25},
        # {"ice_controlling": {"tie_breaker": 0x6569bb58ab364c66}, "_selector": 0x802a},
        {"ice_controlled": {"tie_breaker": 0x6569bb58ab364c66}, "_selector": 0x8029},
        {"priority": {"priority": 0x6e001eff}, "_selector": 0x24},
    ]
    
    msg = public_construct(padding_0_attribute_spec, attributes)
    
    stun_header = {
        "zeroes": 0,
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        # |M |M |M|M|M|C|M|M|M|C|M|M|M|M|
        # |11|10|9|8|7|1|6|5|4|0|3|2|1|0|
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        "message_type": 0b00000000000001,
        "message_length": len(msg)+24,
        "magic_cookie": magic_cookie,
        "transaction_id": random.randrange(1<<96),
    }
    
    msg = public_construct(stun_header_spec, stun_header) + msg
    msg = append_message_integrity(msg, remote_pwd.encode('utf-8'))
    msg = append_fingerprint(msg)
    return msg


In [13]:
try:
    sock.close()
except:
    pass

sock = SocketReader(target_ip, target_port, 'udp')

sock.connect(source_port=sport)

In [14]:
running = True
import time
from queue import Queue

dts_queue = Queue(maxsize=10)

last_received = time.time()

def handler():
    while running:
        resp = sock.read_once()
        if resp[1] == 0xfe and (resp[2] == 0xfd or resp[2] == 0xff):
            dts_queue.put(resp)
        res, rem = public_match(stun_header_spec, resp)
        if res['message_type'].int() != 0b00000000000001:
            continue
        pkg = create_response(res['transaction_id'])
        sock.send(pkg)
        print('+', end='')
        last_received = time.time()

def sender():
    while running:
        sock.send(create_request())
        print('*', end='')
        time.sleep(2 if time.time() - last_received < 6 else 0.2)

import threading
threading.Thread(target=handler).start()
threading.Thread(target=sender).start()

*+++++*+*****+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*****

Exception in thread Thread-6 (sender):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/mie/.virtualenvs/notebooks/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_948445/2590931976.py", line 24, in sender
  File "/home/mie/notebooks/utils.py", line 17, in send
    self.sock.sendall(data)
ConnectionRefusedError: [Errno 111] Connection refused


In [15]:
# running = False
# sock.close()


In [16]:
transcript = b''

In [17]:
handshake_spec = """
8==1?24@client_hello(
    16@version
    -32B@random
    -8@session_id()
    -8@cookie()
    -16@suites(...16)
    -8@compression()
    -16@extensions(
        ...(
            16==23?16@extended_master_secret()
            :16==0xff01?16@renegotiation_info(8@renegotiation_info())
            :16==10?16@ext_supported_curves(
                16@data(...16)
            )
            :16==11?16@ext_supported_points(
                8@supported_points()
            )
            :16==35?16@ext_session_tickets()
            :16==13?16@ext_signature_algorithms(
                16@data(...16)
            )
            :16==14?16@use_srtp(
                16@data(...16)
                -8@mki()
            )
            :16@selector-16@raw_extension()
        )
    )
)
:8==2?24@server_hello(
    16@version
    -32B@random
    -8@session_id()
    -16@suite
    -8@compression
    -16@extensions(
        ...(
            16==23?16@ext_master_secret()
            :16==11?16@ext_supported_points(
                8@supported_points()
            )
            :16==35?16@ext_session_tickets()
            :16==5?16@ext_status_request(
                8@status_type
                -16@responder_id_list
                -16@request_extensions
            )
            :16==0xff01?16@ext_renegotiation_info(
                8@secure_renegotitation()
            )
            :16==14?16@use_srtp(
                16@data(...16)
                -8@mki()
            )
            :16@selector-16@raw_extension()
        )
    )
)
:8==11?24@certificates_data(
    24@certificates(
        ...(
            24@certificate()
        )
    )
)
:8==12?24@server_key_exchange(
    8@curve_type
    -16@named_curve
    -8@pubkey()
    -16@sig_algorithm
    -16@signature()
)
:8==13?24@certificate_request(
    8@cert_types(...(8@cert_type))
    -16@sig_hash_algs(...(16@sig_hash_alg))
    -16@distinguished_names()
)
:8==14?24@server_hello_done()
:8==16?24@client_key_exchange(
    8@pubkey()
)
:8==15?24@certificate_verify(
    16@sig_algorithm
    -16@signature()
)
:8==20?24@finished()
"""

fragment_spec = """
4B@type_and_length
-16@message_sequence
-24@fragment_offset
-24@fragment()
"""

dtls_record_spec = """
8==22?16@tls_version-16@epoch-6B@sequence_number-16@tls_handshake(%s)
:8==20?16@tls_version-16@epoch-6B@sequence_number-16@change_cipher_spec_message()
"""%fragment_spec

In [18]:
msg = dts_queue.get()

In [19]:
a = 'aaa'
a[1:-0]

''

In [20]:
packet, rem = public_match(dtls_record_spec, msg)
transcript += msg[13:]
print(transcript)
packet

b'\x01\x00\x00\x84\x00\x00\x00\x00\x00\x00\x00\x84\xfe\xfdR\xf8\x88\xda,\x91<`\xe9>r\xf04\xae\x8d\xd3\xb2\x06y\x170X)\xad\xe0%\x80\xfc\x17O4!\x00\x00\x00\x16\xc0+\xc0/\xcc\xa9\xcc\xa8\xc0\t\xc0\x13\xc0\n\xc0\x14\x00\x9c\x00/\x005\x01\x00\x00D\x00\x17\x00\x00\xff\x01\x00\x01\x00\x00\n\x00\x08\x00\x06\x00\x1d\x00\x17\x00\x18\x00\x0b\x00\x02\x01\x00\x00#\x00\x00\x00\r\x00\x14\x00\x12\x04\x03\x08\x04\x04\x01\x05\x03\x08\x05\x05\x01\x08\x06\x06\x01\x02\x01\x00\x0e\x00\t\x00\x06\x00\x01\x00\x08\x00\x07\x00'


{'tls_version': 0xfeff,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000000,
 'tls_handshake': {'type_and_length': 0x01000084,
  'message_sequence': 0x0000,
  'fragment_offset': 0x000000,
  'fragment': 0xfefd52f888da2c913c60e93e72f034ae8dd3b2067917305829ade02580fc174f342100000016c02bc02fcca9cca8c009c013c00ac014009c002f00350100004400170000ff01000100000a00080006001d00170018000b0002010000230000000d00140012040308040401050308050501080606010201000e0009000600010008000700}}

In [21]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
client_hello, _ = public_match(handshake_spec, record)
client_hello

{'client_hello': {'version': 0xfefd,
  'random': 0x52f888da2c913c60e93e72f034ae8dd3b2067917305829ade02580fc174f3421,
  'session_id': '',
  'cookie': '',
  'suites': [0xc02b,
   0xc02f,
   0xcca9,
   0xcca8,
   0xc009,
   0xc013,
   0xc00a,
   0xc014,
   0x009c,
   0x002f,
   0x0035],
  'compression': 0x00,
  'extensions': [{'extended_master_secret': '', '_selector': 0x0017},
   {'renegotiation_info': {'renegotiation_info': ''}, '_selector': 0xff01},
   {'ext_supported_curves': {'data': [0x001d, 0x0017, 0x0018]},
    '_selector': 0x000a},
   {'ext_supported_points': {'supported_points': 0x00}, '_selector': 0x000b},
   {'ext_session_tickets': '', '_selector': 0x0023},
   {'ext_signature_algorithms': {'data': [0x0403,
      0x0804,
      0x0401,
      0x0503,
      0x0805,
      0x0501,
      0x0806,
      0x0601,
      0x0201]},
    '_selector': 0x000d},
   {'use_srtp': {'data': [0x0001, 0x0008, 0x0007], 'mki': ''},
    '_selector': 0x000e}]},
 '_selector': 0x01}

In [22]:
dtls_message_encoded = b''

In [23]:
server_hello = {
  'server_hello': {
      'version': 0xfefd,
      'random': bytes.fromhex('2dd93a4dd171b16cef3fd22029ab20c59938934d74b4c597b762f16ff261735b'),
      'session_id': '',
      'suite': 0xc02b,
      'compression': 0x00,
      'extensions': [{'ext_master_secret': '', '_selector': 0x0017},
       {'ext_renegotiation_info': {'secure_renegotitation': ''},
        '_selector': 0xff01},
       {'ext_supported_points': {'supported_points': 0x00}, '_selector': 0x000b},
       {'ext_session_tickets': '', '_selector': 0x0023},
       {'use_srtp': {'data': [0x0001], 'mki': ''}, '_selector': 0x000e}]
  },
  '_selector': 0x02
}

server_hello_encoded = public_construct(handshake_spec, server_hello)



dtls_record = {
    "tls_version": 0xfeff,
    "tls_handshake": {
        'type_and_length': server_hello_encoded[:4],
        'message_sequence': 0,
        'fragment_offset': 0,
        'fragment': server_hello_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 0,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
transcript += dtls_record_encoded[13:]
dtls_message_encoded += dtls_record_encoded


In [24]:

certificate_message = {
'certificates_data': {'certificates': [{'certificate': OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_ASN1, cert)}]},
 '_selector': 11}

certificate_message_encoded = public_construct(handshake_spec, certificate_message)


dtls_record = {
    "tls_version": 0xfeff,
    "tls_handshake": {
        'type_and_length': certificate_message_encoded[:4],
        'message_sequence': 1,
        'fragment_offset': 0,
        'fragment': certificate_message_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 1,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
transcript += dtls_record_encoded[13:]
dtls_message_encoded += dtls_record_encoded

In [25]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey, X25519PublicKey

server_private_key = X25519PrivateKey.generate()
server_public_key = server_private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)


In [26]:
server_key_sign_spec = """32B@client_random-32B@server_random-8@curve_type-16@named_curve-8@pubkey()"""
server_key_sign = {
    "client_random": client_hello['client_hello']['random'],
    "server_random": server_hello['server_hello']['random'],
    "curve_type": 3,
    "named_curve": 0x001d,
    "pubkey": server_public_key,
    "sig_algorithm": 0x0403,
    "_selector": 12,
}

enc = public_construct(server_key_sign_spec, server_key_sign)

server_key_sign['signature'] = own_cert_private_key.sign(enc, ec.ECDSA(hashes.SHA256()))

server_key_sign_encoded = public_construct(handshake_spec, server_key_sign)



dtls_record = {
    "tls_version": 0xfeff,
    "tls_handshake": {
        'type_and_length': server_key_sign_encoded[:4],
        'message_sequence': 2,
        'fragment_offset': 0,
        'fragment': server_key_sign_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 2,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
transcript += dtls_record_encoded[13:]
dtls_message_encoded += dtls_record_encoded

In [27]:
cert_req = {'certificate_request': {'cert_types': [{'cert_type': 0x01},
   {'cert_type': "@"}],
  'sig_hash_algs': [{'sig_hash_alg': 0x0403},
   {'sig_hash_alg': 0x0804},
   {'sig_hash_alg': 0x0401},
   {'sig_hash_alg': 0x0503},
   {'sig_hash_alg': 0x0805},
   {'sig_hash_alg': 0x0501},
   {'sig_hash_alg': 0x0806},
   {'sig_hash_alg': 0x0601},
   {'sig_hash_alg': 0x0201}],
  'distinguished_names': ''},
 '_selector': 13}

cert_req_encoded = public_construct(handshake_spec, cert_req)


dtls_record = {
    "tls_version": 0xfeff,
    "tls_handshake": {
        'type_and_length': cert_req_encoded[:4],
        'message_sequence': 3,
        'fragment_offset': 0,
        'fragment': cert_req_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 3,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
transcript += dtls_record_encoded[13:]
dtls_message_encoded += dtls_record_encoded

In [28]:
server_hello_done = {'server_hello_done': '', '_selector': 0x0e}
server_hello_done_encoded = public_construct(handshake_spec, server_hello_done)


dtls_record = {
    "tls_version": 0xfeff,
    "tls_handshake": {
        'type_and_length': server_hello_done_encoded[:4],
        'message_sequence': 4,
        'fragment_offset': 0,
        'fragment': server_hello_done_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 4,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
transcript += dtls_record_encoded[13:]
dtls_message_encoded += dtls_record_encoded

In [29]:
sock.send(dtls_message_encoded)

In [30]:
msg = dts_queue.get()
msg

b"\x16\xfe\xfd\x00\x00\x00\x00\x00\x00\x00\x01\x01+\x0b\x00\x01\x1f\x00\x01\x00\x00\x00\x00\x01\x1f\x00\x01\x1c\x00\x01\x190\x82\x01\x150\x81\xbc\xa0\x03\x02\x01\x02\x02\x08\x7f\x02\xa3\xf4\x19\xcc`\xac0\n\x06\x08*\x86H\xce=\x04\x03\x020\x111\x0f0\r\x06\x03U\x04\x03\x0c\x06WebRTC0\x1e\x17\r240617204224Z\x17\r240718204224Z0\x111\x0f0\r\x06\x03U\x04\x03\x0c\x06WebRTC0Y0\x13\x06\x07*\x86H\xce=\x02\x01\x06\x08*\x86H\xce=\x03\x01\x07\x03B\x00\x04\xb1\xdd\xf6C\x18\x12g\xa5\xfby\xb8\xa7\x95\xe0\xb5\xecULs\r\xeaT\x9b,\xc7\xb3\xdcb\x12t\xe9\xfdqeC\xa7\xa1\xb4\xe7\x19\xa6\t\xcf\xc0\xd31\xdf~48\x00z\x0b\xcbcn\xf3O\x16\xa4\x04R\xbce0\n\x06\x08*\x86H\xce=\x04\x03\x02\x03H\x000E\x02 `\xc0\xdf\xb3\xed\x0f\xfc\xfb')\x12\x18\xb55\x81\xc9\x918\x93\x8d*:\x81-qJ;\xc9\xd4\xee\xad\xb1\x02!\x00\xb3\x13\x8e\xf3\x93b\xbc\x90\x83\xb7\xca\x15\x14\xdd\n\xc6\\\x07\xc4\xfb\x9eM\xea\x1fg\xd5\xebe\xad\xcb\x04\x91\x16\xfe\xfd\x00\x00\x00\x00\x00\x00\x00\x02\x00-\x10\x00\x00!\x00\x02\x00\x00\x00\x00\x00! \x0e\x00\x8a>\

In [31]:
packet, rem = public_match(dtls_record_spec, msg)
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000001,
 'tls_handshake': {'type_and_length': 0x0b00011f,
  'message_sequence': 0x0001,
  'fragment_offset': 0x000000,
  'fragment': 0x00011c000119308201153081bca00302010202087f02a3f419cc60ac300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303631373230343232345a170d3234303731383230343232345a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d03010703420004b1ddf643181267a5fb79b8a795e0b5ec554c730dea549b2cc7b3dc621274e9fd716543a7a1b4e719a609cfc0d331df7e3438007a0bcb636ef34f16a40452bc65300a06082a8648ce3d0403020348003045022060c0dfb3ed0ffcfb27291218b53581c99138938d2a3a812d714a3bc9d4eeadb1022100b3138ef39362bc9083b7ca1514dd0ac65c07c4fb9e4dea1f67d5eb65adcb0491}}

In [32]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
peer_cert, _ = public_match(handshake_spec, record)
peer_cert

{'certificates_data': {'certificates': [{'certificate': 0x308201153081bca00302010202087f02a3f419cc60ac300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303631373230343232345a170d3234303731383230343232345a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d03010703420004b1ddf643181267a5fb79b8a795e0b5ec554c730dea549b2cc7b3dc621274e9fd716543a7a1b4e719a609cfc0d331df7e3438007a0bcb636ef34f16a40452bc65300a06082a8648ce3d0403020348003045022060c0dfb3ed0ffcfb27291218b53581c99138938d2a3a812d714a3bc9d4eeadb1022100b3138ef39362bc9083b7ca1514dd0ac65c07c4fb9e4dea1f67d5eb65adcb0491}]},
 '_selector': "
 "}

In [33]:
if 'certificates_data' not in peer_cert:
    raise RuntimeError("repeated package, retry from dts_queue.get()")
transcript += msg[13:-len(rem)]

In [34]:
first_client_cert = None
for cert in peer_cert['certificates_data']['certificates']:
    x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_ASN1, cert['certificate'].bytes())
    if first_client_cert is None:
        first_client_cert = x509
    print(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_TEXT, x509).decode('utf-8')) 

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number: 9152057661570441388 (0x7f02a3f419cc60ac)
        Signature Algorithm: ecdsa-with-SHA256
        Issuer: CN=WebRTC
        Validity
            Not Before: Jun 17 20:42:24 2024 GMT
            Not After : Jul 18 20:42:24 2024 GMT
        Subject: CN=WebRTC
        Subject Public Key Info:
            Public Key Algorithm: id-ecPublicKey
                Public-Key: (256 bit)
                pub:
                    04:b1:dd:f6:43:18:12:67:a5:fb:79:b8:a7:95:e0:
                    b5:ec:55:4c:73:0d:ea:54:9b:2c:c7:b3:dc:62:12:
                    74:e9:fd:71:65:43:a7:a1:b4:e7:19:a6:09:cf:c0:
                    d3:31:df:7e:34:38:00:7a:0b:cb:63:6e:f3:4f:16:
                    a4:04:52:bc:65
                ASN1 OID: prime256v1
                NIST CURVE: P-256
    Signature Algorithm: ecdsa-with-SHA256
    Signature Value:
        30:45:02:20:60:c0:df:b3:ed:0f:fc:fb:27:29:12:18:b5:35:
        81:c9:91:38:93:8d:2a:3a:81:

In [35]:
first_client_cert.digest('sha256')
# TODO: assert digest

b'EB:67:CB:B8:04:53:FA:94:41:77:1F:4D:E1:A4:4E:05:A7:DB:94:67:D4:2C:31:A1:F2:7B:B6:DC:3E:B7:7B:D8'

In [36]:
packet, rem1 = public_match(dtls_record_spec, rem)
transcript += rem[13:-len(rem1)]
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000002,
 'tls_handshake': {'type_and_length': 0x10000021,
  'message_sequence': 0x0002,
  'fragment_offset': 0x000000,
  'fragment': 0x200e008a3eea7a856200d2bde1545b1dbe18dd0af4ba8c14ae25a175300edf6d0d}}

In [37]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
client_key_exchange, _ = public_match(handshake_spec, record)
client_key_exchange

{'client_key_exchange': {'pubkey': 0x0e008a3eea7a856200d2bde1545b1dbe18dd0af4ba8c14ae25a175300edf6d0d},
 '_selector': 0x10}

In [38]:
packet, rem2 = public_match(dtls_record_spec, rem1)
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000003,
 'tls_handshake': {'type_and_length': 0x0f00004b,
  'message_sequence': 0x0003,
  'fragment_offset': 0x000000,
  'fragment': 0x040300473045022100c3350b6990687ad15bc5301a2fa5924e0d4701d3a27ccf159875d13d6b1a5c7f022036047e063f3bce7b2faa7f8f01df45fe99ed31cbb72a07d0e6559ec936ae3414}}

In [39]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
certificate_verify, _ = public_match(handshake_spec, record)
certificate_verify

{'certificate_verify': {'sig_algorithm': 0x0403,
  'signature': 0x3045022100c3350b6990687ad15bc5301a2fa5924e0d4701d3a27ccf159875d13d6b1a5c7f022036047e063f3bce7b2faa7f8f01df45fe99ed31cbb72a07d0e6559ec936ae3414},
 '_selector': 0x0f}

In [40]:
first_client_cert.get_pubkey().to_cryptography_key().verify(
    certificate_verify['certificate_verify']['signature'].bytes(),
    transcript,
    ec.ECDSA(hashes.SHA256()),
)

In [41]:
# transcript += rem1[13:-len(rem2)]
# hhh = rem1[13:-len(rem2)]
# transcript = transcript[:-len(hhh)]

In [42]:
packet, rem3 = public_match(dtls_record_spec, rem2)
packet

{'tls_version': 0xfefd,
 '_selector': 0x14,
 'epoch': 0x0000,
 'sequence_number': 0x000000000004,
 'change_cipher_spec_message': 0x01}

In [43]:
import re
handshake_spec2 = re.sub(r'^(:?8.+\()', r'\1 16@ms-24@fo-24@fl-', handshake_spec, count=0, flags=re.M)
# print(handshake_spec2)
asd = bytearray(transcript)
while asd:
    asd = bytearray(asd)
    asd[3] += 8
    rec, asd = public_match(handshake_spec2, asd)
    print('--------------')
    print(rec)

--------------
{'client_hello': {'ms': 0x0000, 'fo': 0x000000, 'fl': 0x000084, 'version': 0xfefd, 'random': 0x52f888da2c913c60e93e72f034ae8dd3b2067917305829ade02580fc174f3421, 'session_id': '', 'cookie': '', 'suites': [0xc02b, 0xc02f, 0xcca9, 0xcca8, 0xc009, 0xc013, 0xc00a, 0xc014, 0x009c, 0x002f, 0x0035], 'compression': 0x00, 'extensions': [{'extended_master_secret': '', '_selector': 0x0017}, {'renegotiation_info': {'renegotiation_info': ''}, '_selector': 0xff01}, {'ext_supported_curves': {'data': [0x001d, 0x0017, 0x0018]}, '_selector': 0x000a}, {'ext_supported_points': {'supported_points': 0x00}, '_selector': 0x000b}, {'ext_session_tickets': '', '_selector': 0x0023}, {'ext_signature_algorithms': {'data': [0x0403, 0x0804, 0x0401, 0x0503, 0x0805, 0x0501, 0x0806, 0x0601, 0x0201]}, '_selector': 0x000d}, {'use_srtp': {'data': [0x0001, 0x0008, 0x0007], 'mki': ''}, '_selector': 0x000e}]}, '_selector': 0x01}
--------------
{'server_hello': {'ms': 0x0000, 'fo': 0x000000, 'fl': 0x000044, 'vers

In [44]:
encrypted_dtls_record_spec = """
8@content_type-16@tls_version-16@epoch-6B@sequence_number-16@encrypted_data()
"""

In [45]:
packet, rem4 = public_match(encrypted_dtls_record_spec, rem3)
# transcript += rem3[13:-len(rem4)]
packet

{'content_type': 0x16,
 'tls_version': 0xfefd,
 'epoch': 0x0001,
 'sequence_number': 0x000000000000,
 'encrypted_data': 0x0001000000000000ead81d41bee8cc3e1481c41047c361399eea42a91a1b6d80fdb95e60352d7f0e80fae53321ab7b1c}

In [46]:
from cryptography.hazmat.primitives import hashes, hmac

def HMAC_SHA256(secret, val):
    h = hmac.HMAC(secret, hashes.SHA256())
    h.update(val)
    return h.finalize()

def P_SHA256(secret, seed, l):
    b = bytearray()
    A = HMAC_SHA256(secret, seed)
    while len(b) < l:
        b += HMAC_SHA256(secret, A + seed)
        A = HMAC_SHA256(secret, A)
    return b[:l]

def PRF(secret, label, seed, l):
    return P_SHA256(secret, label + seed, l)

In [47]:
def calculate_sha256(data):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(data)
    return digest.finalize()

In [48]:
shared_key = server_private_key.exchange(X25519PublicKey.from_public_bytes(client_key_exchange['client_key_exchange']['pubkey'].bytes()))
print("calculated shared key", shared_key)

calculated shared key b'\x14\x12\x08\xe9R\xdb\x9c\x92[\xff\xf0[\xfa_$\xb4!,\xd8\x0f\x88\xf1\x1a&\x91\x9b\x84\xd1\xb1=@b'


In [49]:
master_key = PRF(
    shared_key,
    b"extended master secret",
    # client_hello['client_hello']['random'].bytes() + server_hello['server_hello']['random'],
    calculate_sha256(transcript),
    48,
)
master_key.hex()

'1c16fe213b147ace798e1e19f910a8a8e4c8247fe2192f5bfa071caa43aa709232e30260f524309d788185ad0ebf6322'

In [50]:
keyLen = 16
ivLen  = 4

raw_keys = PRF(
    master_key,
    b"key expansion",
    server_hello['server_hello']['random'] + client_hello['client_hello']['random'].bytes(),
    40,
)

client_write_key = raw_keys[:keyLen]
raw_keys = raw_keys[keyLen:]

server_write_key = raw_keys[:keyLen]
raw_keys = raw_keys[keyLen:]

client_write_iv = raw_keys[:ivLen]
raw_keys = raw_keys[ivLen:]

server_write_iv = raw_keys[:ivLen]

In [58]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

aad_spec = """16@epoch-6B@sequence_number-8@content_type-16@tls_version-16@data_length"""

def decrypt_packet(packet, key, iv):
    encrypted = packet['encrypted_data']
    if not isinstance(encrypted, bytes):
        encrypted = encrypted.bytes()

    aad = packet.copy()
    aad['data_length'] = len(encrypted) - 8 - 16
    aad_encoded = bytearray(public_construct(aad_spec, aad))
    aad_encoded[:8] = encrypted[:8]

    cipher = AESGCM(key)
    all_server_cert_packet_encoded = cipher.decrypt(iv + encrypted[:8], encrypted[8:], aad_encoded)
    print(all_server_cert_packet_encoded)
    all_server_cert_packet, _ = public_match(fragment_spec, all_server_cert_packet_encoded)
    packet['tls_handshake'] = all_server_cert_packet
    return packet
    

def encrypt_packet(packet, key, iv):
    to_encrypt = public_construct(fragment_spec, packet['tls_handshake'])

    aad = packet.copy()
    aad['data_length'] = len(to_encrypt)
    # aad['sequence_number'] = aad['tls_handshake']['message_sequence']
    aad_encoded = public_construct(aad_spec, aad)
    cipher = AESGCM(key)
    encrypted_app_data = cipher.encrypt(iv + aad_encoded[:8], to_encrypt, aad_encoded)
    packet['encrypted_data'] = aad_encoded[:8] + encrypted_app_data
    packet['data_length'] = len(packet['encrypted_data'])
    return packet

In [59]:
# print(len(client_write_iv + packet['epoch'].bytes()+packet['sequence_number'].bytes()))
finished_fragment = decrypt_packet(
    packet,
    client_write_key,
    client_write_iv,
)
finished_fragment
print(finished_fragment)


b'\x14\x00\x00\x0c\x00\x04\x00\x00\x00\x00\x00\x0c\x00u\x7f\x1fy\xa5#\n\x05v\xd3_'
{'content_type': 0x16, 'tls_version': 0xfefd, 'epoch': 0x0001, 'sequence_number': 0x000000000000, 'encrypted_data': 0x0001000000000000ead81d41bee8cc3e1481c41047c361399eea42a91a1b6d80fdb95e60352d7f0e80fae53321ab7b1c, 'tls_handshake': {'type_and_length': 0x1400000c, 'message_sequence': 0x0004, 'fragment_offset': 0x000000, 'fragment': 0x00757f1f79a5230a0576d35f}}


In [53]:
record = finished_fragment['tls_handshake']['type_and_length'].bytes() + finished_fragment['tls_handshake']['fragment'].bytes()
finished_msg, _ = public_match(handshake_spec, record)
finished_msg['finished'].bytes().hex()

'00757f1f79a5230a0576d35f'

In [54]:
# Add certificate verify message
transcript += rem1[13:-len(rem2)]

In [55]:
client_verify_data = PRF(master_key, b"client finished", calculate_sha256(transcript), 12)
client_verify_data.hex()

'00757f1f79a5230a0576d35f'

In [56]:
assert client_verify_data == finished_msg['finished'].bytes(), "client verify data mismatch"

In [57]:
encrypted_dtls_record = {
    "tls_version": 0xfeff,
    "epoch": 1,
    "sequence_number": 0,
    "handshake_data": bytes(finished),
    "_selector": 22
}
encrypted_dtls_record_encoded = public_construct(encrypted_dtls_record_spec, encrypted_dtls_record)
# transcript += dtls_record_encoded[13:]
dtls_message_encoded += encrypted_dtls_record_encoded

NameError: name 'finished' is not defined

In [ ]:
len(aad['handshake_data'].bytes()) - 16

In [ ]:
cipher = AESGCM(client_write_key)
cipher.encrypt?